In [1]:
import cProfile
import lysis
import math
import os
import numpy as np

In [2]:
e = lysis.util.Experiment(os.path.join("..", "..", "data"), experiment_code='2023-01-30-1700')

In [3]:
p = {"rows": 12,
         "cols": 9,
         "empty_rows": 3,
         "total_molecules": 430,
         "total_time": 10*60,
         "duplicate_fortran": True
         }
e.initialize_macro_param(p)
# e.read_file()

In [14]:
e.to_file()

In [4]:
macro = lysis.MacroscaleRun(e)

In [5]:
macro.run()

  0%|          | 0/1752366 [00:00<?, ?it/s]

In [6]:
macro.bind_info_file.close()

In [10]:
#n = np.empty((0,), dtype=np.float_)
with open(os.path.join(macro.exp.os_path, "m_bind_t.p-array.dat"), 'r') as file:
    n = np.fromregex(file, r'(\-?\d\.\d*e?\-?\+?\d*)\w*(\-?\d\.\d*e?\-?\+?\d*)\w*(\-?\d\.\d*e?\-?\+?\d*)', dtype=[('r1', np.float_), ('t', np.float_), ('bind', np.float_)])
n

array([], dtype=[('r1', '<f8'), ('t', '<f8'), ('bind', '<f8')])

In [19]:
np.array2string(n)[1:-1]

'9.50116062e-43 9.15604175e-72 1.39804328e-76 ... 5.06869768e-86\n 5.64341326e-62 5.39365393e-62'

In [7]:
current_time = 2920 * macro.exp.macro_params.time_step
current_time

0.9997907233333332

In [9]:
macro.m_fiber_status = macro.fiber_status[macro.location]
macro.unbind_by_degradation(macro.bound & (macro.m_fiber_status < current_time), current_time)
macro.unbind_by_time(macro.bound & (macro.binding_time < current_time), current_time)

should_bind = (~macro.bound
               & (macro.binding_time < current_time)
               & (macro.m_fiber_status > current_time)
               & (macro.waiting_time < current_time))

move_chance = np.ones(macro.exp.macro_params.total_molecules, dtype=np.double)
move_chance[~macro.bound] = macro.rng.random(np.count_nonzero(~macro.bound), dtype=np.double)
should_move = move_chance < macro.exp.macro_params.moving_probability
conflict = should_bind & should_move
threshold = np.full(macro.exp.macro_params.total_molecules, -1, dtype=np.double)
threshold[conflict] = ((current_time - macro.binding_time[conflict])
                       / macro.exp.macro_params.time_step)
should_bind[conflict] = macro.rng.random(np.count_nonzero(conflict)) > threshold[conflict]
should_move[conflict] = ~should_bind[conflict]

macro.bind(should_bind, current_time)

In [10]:
still_stuck_to_fiber = np.full(macro.exp.macro_params.total_molecules, False, dtype=np.bool_)
still_stuck_to_fiber[should_bind] = macro.waiting_time[should_bind] > current_time
np.logical_and(still_stuck_to_fiber[should_bind], macro.unbound_by_degradation[should_bind], out=still_stuck_to_fiber[should_bind])
np.count_nonzero(still_stuck_to_fiber)

0

In [11]:
for n in np.arange(macro.exp.macro_params.total_molecules)[still_stuck_to_fiber]:
    print(n)

In [15]:
a = 499
b = 500
c = a*e.macro_params.time_step
d = b*e.macro_params.time_step

In [16]:
%timeit a < b

36.1 ns ± 0.269 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [17]:
%timeit c < d

33.1 ns ± 0.401 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [ ]:
lysis.from_fortran_edge_index(51, 4, 5)

In [3]:
rng = np.random.default_rng(seed=489489496)

In [5]:
a = rng.random(60).reshape(6, 10)

In [7]:
b = rng.random(10)
c = b < 0.5

In [8]:
c

array([ True, False, False,  True, False,  True, False,  True,  True,
        True])

In [10]:
a[5][c]

array([0.46961349, 0.51494345, 0.93896747, 0.49600301, 0.5810864 ,
       0.22792777])

In [11]:
a[5]

array([0.46961349, 0.92091926, 0.20134264, 0.51494345, 0.99789049,
       0.93896747, 0.76984013, 0.49600301, 0.5810864 , 0.22792777])

In [50]:
c

array([[False, False,  True,  True],
       [False, False,  True,  True],
       [False,  True,  True,  True]])

In [48]:
np.take_along_axis(a[b], c[:,::-1], axis=1)

array([[0.28751422, 0.17398308, 0.55931457, 0.53022389],
       [0.34155978, 0.28751422, 0.51085294, 0.95012645],
       [0.24432482, 0.24990649, 0.08039694, 0.63829939]])

In [73]:
d = np.count_nonzero(c, axis=1)
d

array([2, 2, 3], dtype=int64)

In [71]:
e = d[np.newaxis, :]
np.ix_(,d)

(array([[0]], dtype=int64), array([[2, 2, 3]], dtype=int64))

In [74]:
a[b][np.full(3, True), d]

array([0.28751422, 0.51085294, 0.63829939])

In [81]:
%timeit np.any(a < 0.1)

8.26 µs ± 108 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [83]:
a = np.empty((20, 9), dtype=int)

In [84]:
a[:] = np.arange(9)

In [85]:
a

array([[0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8]])

In [86]:
a[4,6]=100
a

array([[  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5, 100,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3, 

In [3]:
tot_sec = 6110*10
tot_sec // (60*60), (tot_sec // 60) % 60, tot_sec % 60

(16, 58, 20)

In [10]:
import dataclasses
import inspect

import numpy as np

import lysis

In [9]:
sig = inspect.signature(lysis.util.MacroParameters)

In [10]:
for key in sig.parameters:
    print(key, sig.parameters[key].default)

binding_rate 0.1
pore_size 0.00010135
diffusion_coeff 5e-07
binding_sites 427
forced_unbind 0.0852
average_bind_time 27.8
grid_node_distance 1.0862
cols 93
rows 121
empty_rows 28
total_molecules 43074
moving_probability 0.2
microscale_runs 50000
total_trials 10
total_time 1200
seed -2137354075
state (129281, 362436069, 123456789, None)
save_interval 10
macro_version diffuse_into_and_along
log_lvl 30
duplicate_fortran False
processing_library numpy


In [18]:
inspect.getdoc(e.macro_params.binding_rate)

'Convert a string or number to a floating point number, if possible.'

In [20]:
lysis.util.MacroParameters.binding_rate.__doc__

'Convert a string or number to a floating point number, if possible.'

In [6]:
a = e.macro_params.fortran_names()

In [4]:
a = lysis.util.FortranMacro(e, "../..", "bin/macro-normal", ".dat", ".f-normal.dat")

In [6]:
a.run()

['bin/macro-normal', '--expCode', '2023-01-30-1700', '--inFileCode', '.dat', '--outFileCode', '.f-normal.dat', '--N', '9', '--F', '12', '--Ffree', '2', '--M', '430', '--tf', '600']
None


In [9]:
"binding_rate" in a

True

In [8]:
a

{'binding_rate': 'kon',
 'pore_size': 'delx',
 'diffusion_coeff': 'Diff',
 'binding_sites': 'bs',
 'forced_unbind': 'frac_forced',
 'average_bind_time': 'avgwait',
 'grid_node_distance': 'dist',
 'cols': 'N',
 'rows': 'F',
 'total_edges': 'num',
 'empty_rows': 'Ffree-1',
 'last_empty_edge': 'enoFB-1',
 'total_molecules': 'M',
 'moving_probability': 'q',
 'microscale_runs': 'nummicro',
 'total_trials': 'stats',
 'total_time': 'tf',
 'time_step': 'tstep',
 'total_time_steps': 'num_t',
 'seed': 'seed',
 'state': 'state',
 'number_of_saves': 'nplt'}

In [7]:
s = "Ffree-1"
s[:-2]

'Ffree'